<a href="https://colab.research.google.com/github/mptorpo/analitica_negocios/blob/main/Reto2_MiguelPimientoRestrepo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RETO 2 - ÁRBOLES DE DECISIÓN: Tech Sales Report**

**Descripción de Variables**

* Business_Num: Uno de los dos grupos de productos = Hardware (0) y Software (1).
* Age: Edad del empleado
* Female: Si es mujer o no (1=Si es mujer | 0=Caso contrario)
* Years: El número de años que el empleado ha estado en la empresa
* Certificates: El número de certiﬁcaciones profesionales relevantes que ha obtenido cada empleado.
* Feedback: El puntaje de retroalimentación promedio que cada empleado recibe de sus compañeros y supervisor en la evaluación anual de 360 grados. De 0 a 4.
* Salary: Sueldo base anual de cada empleado.
* NPS: Net Promoter Score (medición de la satisfacción y lealtad del cliente)
* College: Si el empleado tiene o no un título universitario de cuatro años (1=Si | 0=No).

0. Se procede con la carga de las librerias de trabajo

In [ ]:
import numpy as np
import pandas as pd

#Para implementación y evaluación del modelo
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Para la carga automática de los archivos desde el drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Se procede con la carga de la base de datos

In [ ]:
nxl='/content/drive/MyDrive/Colab Notebooks/2. TechSalesReport_Int_L.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0)

#Cambiar las variables de texto a variables numericas (Business y College)
mapeo_business = {'Hardware': 0, 'Software': 1} #Mapear los valores
XDB['Business_Num'] = XDB['Business'].map(mapeo_business) # Método 'map' para aplicar el mapeo a la columna 'Business'

mapeo_college = {'No': 0, 'Yes': 1} #Mapear los valores
XDB['College_Num'] = XDB['College'].map(mapeo_college) # Método 'map' para aplicar el mapeo a la columna 'College'

#Seleccionamos las variables
XD=XDB[['Business_Num','Age','Female','Years','Certificates','Feedback','Salary','NPS']] #Aquí estan las  8 variables de entrada
yd=XDB['College_Num'] #Aquí esta la variable principal de referencia

2. Caracterización de la base de datos (total de personas, número de personas inscritas y número de personas no escritas)

In [ ]:
from sklearn.naive_bayes import GaussianNB #Cargar GaussianNB

mnb=GaussianNB() #Aplicar Naive Bayes
mnb.fit(XD,yd) #Aplicar Naive Bayes teniendo en cuenta variables

#Se obtiene la info del modelo
ct=mnb.class_count_
print("Como están separados los datos (Sin Título | Con Título):", ct) #Separación: Titulo Universitario | Sin Titulo
print("El total de los datos/personas es:", len(XD)) #Total Datos
print("Los datos porcentualmente estan divididos (Sin Título | Con Título):", ct/len(XD)) #División % (Sin Título| Con Título)

#Sin Título: (0 = 4470) | Con Título: 1 = 17520

Como están separados los datos (Sin Título | Con Título): [ 4470. 17520.]
El total de los datos/personas es: 21990
Los datos porcentualmente estan divididos (Sin Título | Con Título): [0.20327422 0.79672578]


3. Se procede con la implementación del modelo + visualización del árbol de decisión

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd)

#Se procede con el despliegue del árbol de decisión
from sklearn.tree import export_graphviz #Exportar
from pydotplus import graph_from_dot_data #Crear el Gráfico

#Variables
vs=['Business_Num','Age','Female','Years','Certificates','Feedback','Salary','NPS']

#Imagen del árbol de decisión
dot_data=export_graphviz(mar,feature_names=vs)
graph=graph_from_dot_data(dot_data)
graph.write_png('1. ArbolCollege.png')

#Nodos mas importantes/relevantes: Gini en 0 = mas certeros (100%) = Nodos Puros
#Sirven para tener certeza de quienes tener en cuenta o cuales no

True

4. Pronóstico base de datos "pronóstico" + informe de pronóstico en un archivo Excel

In [ ]:
#Se toma la totalidad de los solicitantes
XDB2=pd.read_excel(nxl,sheet_name=1)
yd2=XDB2[['College']]

#Donde están los que no poseen infromación
filas=np.where((np.isnan(yd2))==True)[0]
print("Las filas que no poseen información:",filas)

print( ) #Para la estetica

#Asegurar que las variables "Business" y "College" se mapeen a valores numéricos
XDB2['Business_Num'] = XDB2['Business'].map({'Hardware': 0, 'Software': 1})
XDB2['College_Num'] = XDB2['College'].map({'No': 0, 'Yes': 1})

#Se hace el pronóstico de la PreApr para estas personas
XD2=XDB2[['Business_Num','Age','Female','Years','Certificates','Feedback','Salary','NPS']]
XD2=XD2.iloc[filas,]

#Se hace el pronostico para estas personas
ydp2=mar.predict(XD2)
print("Las predicciones son:",ydp2)

print( ) #Para la estetica

#Crear el informe con la PreApr
dfp=pd.DataFrame(np.column_stack((filas,XD2,ydp2)))
dfp.columns=['filas','Business_Num','Age','Female','Years','Certificates','Feedback','Salary','NPS','College_Num']
dfp.to_excel('InformeCollegePro.xlsx')

print("El total de pronosticados a no tener título Universitario:",len(np.where(ydp2==0)[0]))
print("El total de pronosticados a si tener título Universitario:",len(np.where(ydp2==1)[0]))

Las filas que no poseen información: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]

Las predicciones son: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

El total de pronosticados a no tener título Universitario: 0
El total de pronosticados a si tener título Universitario: 16


5. Desempeño del modelo

In [ ]:
ydp=mar.predict(XD)
cm=confusion_matrix(yd,ydp)
print(cm)

VN=cm[0,0]; FP=cm[0,1]; FN=cm[1,0]; VP=cm[1,1]

print( ) #Para la estetica

Exactitud=(VP+VN)/(VP+VN+FP+FN) #Funcionamiento general del modelo
print('La exactitud es:',Exactitud)

Sensibilidad=VP/(VP+FN) #Como se comporto el modelo con que tienen titulo
print('La sensibilidad es:',Sensibilidad)

Especificidad=VN/(VN+FP) #Como se comporto el modelo con los que no tienen titulo
print('La especificidad es:',Especificidad)

Precision=VP/(VP+FP) #Cual es el acierto clasificando los Positivos
print('La precisión es:',Precision)

TasaError=(FP+FN)/(VP+VN+FP+FN) #Equivocación general del modelo
print('La tasa de error es:',TasaError)

PredNeg=VN/(VN+FN) #Como funciona el modelo para los 0
print('La tasa de predicción negativa es:',PredNeg)

[[  114  4356]
 [   69 17451]]

La exactitud es: 0.7987721691678036
La sensibilidad es: 0.9960616438356165
La especificidad es: 0.025503355704697986
La precisión es: 0.8002476269087908
La tasa de error es: 0.20122783083219645
La tasa de predicción negativa es: 0.6229508196721312


**Guía (TechSalesReport)**

1. El archivo Google Colab, llevar a cabo la descripción de cada una de las variables de acuerdo con el libro guía, esto con el fin de dar mayor contexto al análisis:
- Ver "Descripción de Variables" antes del punto 0.

-

2. Tomar máximo 4 niveles de análisis para la estructura del árbol de decisión:
- Ver png ('1. ArbolCollege.png') que genera el codigo del punto 3 como visualización.

-

3. Llevar a cabo la caracterización de la base de datos, total de personas, número de personas inscritas y número de personas no escritas:
- Total de Personas: 21990.
- Número de Personas con Título: 17520.
- Número de Personas sin Título: 4470.
- División %: Sin Título = 20.33% | Con Título = 79.67%.
- Para ver procedimiento ver el punto 2.

-

4. Determinar el número de nodos puros para el árbol asociado con la estructura del modelo:
- 0, de nuevo se puede evidenciar en el punto 3, en la visualización del árbol de decisió.

-

5. Llevar a cabo el pronóstico del título Universitario para las personas agrupadas en la base de datos de pronóstico:
- El total de pronósticados a no tener título Universitario: 0.
- El total de pronósticados a si tener título Universitario: 16.
- Para ver procedimiento referirce al punto 4.

-

6. Determinar las métricas de desempeño del modelo mediante la utilización de la matriz de confusión sobre el total de los datos:
- Referirse a punto 5 para ver el procedimiento y la matriz de confusión.
- La exactitud es: 79.9%
- La sensibilidad es: 99.6%
- La especificidad es: 2.6%
- La precisión es: 80%
- La tasa de error es: 20.1%
- La tasa de predicción negativa es: 62.3%

-

7. Crear los informes de pronóstico en un archivo de Excel:
- Ver Excel ('InformeCollegePro.xlsx') que genera el codigo del punto 4.

-

**Análisis General:** El modelo de Árbol de Decisión fue construido utilizando una base de datos de 21990 empleados, en donde el 79.67% (17520 empleados) tenían título universitario, mientras que el 20.33% (4470 empleados) no contaban con este. Para la construcción del modelo, se utilizaron ocho variables socioeconómicas (grupos de producto (hardware o software), edad, si son mujeres o no, años de experiencia, certificaciones, retroalimentación, salario base anual, y puntaje NP), excluyendo la variable de personalidad. Se identificaron los nodos puros dentro del árbol (0 en total, de acuerdo a los 4 niveles de análisis que se vieron en el árbol).

El modelo alcanzó una exactitud del 79.9%, superando el umbral del 75%, lo que señala un rendimiento óptimo, y podría ser utilizado para futuras aplicaciones en la gestión del talento y toma de decisiones estratégicas. Además, la sensibilidad fue del 99.6%, lo que indica que el modelo identificó correctamente a la mayoría de los empleados con título universitario. La especificidad, encargada de identificar a los empleados sin título, se situó en 2.5%, lo que muestra un rendimiento muchisimo menor en esta area. En términos de precisión (empleados con título universitario) tuvo un 80%, mientras que la clase negativa (empleados sin título universitario) alcanzó un 62.3%. Además, también vale la pena aclarar que la equivocacion general del modelo (tasa de error) fue de 20.1%. Finalmente, al realizar el pronóstico para los empleados sin información en la base de datos (tab = 1 'Pronóstico'), se determinó que los 16 empleados sin datos fueron clasificados como poseedores de un título universitario (1).